In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/co2-emissions-datamodified/TRAIN.csv
/kaggle/input/co2-emissions-datamodified/TEST.csv
/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv
/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv
/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv


In [31]:
train1 = pd.read_csv('/kaggle/input/co2-emissions-datamodified/TRAIN.csv')
test1 = pd.read_csv('/kaggle/input/co2-emissions-datamodified/TEST.csv')
sample_sub = pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [63]:
#sample_sub

In [21]:
train1.columns

Index(['Country Name', 'year', 'CO2 emissions (metric tons per capita)'], dtype='object')

In [41]:
countries = train1['Country Name'].unique().tolist()

In [33]:
train1.head()

,Country Name,year,CO2 emissions (metric tons per capita)
0,Afghanistan,2000,0.055167
1,Afghanistan,2001,0.055293
2,Afghanistan,2002,0.066810
3,Afghanistan,2003,0.073005
4,Afghanistan,2004,0.054867


In [34]:
train1 = train1.rename(columns = {'CO2 emissions (metric tons per capita)': 'y', 'year': 'ds'})

In [25]:
np.max(train1.y)

47.65696201

In [39]:
train1.isnull().sum()

Country Name      0
ds                0
y               432
dtype: int64

In [35]:
y_avg = np.mean(train1['y']) / 2

In [36]:
y_avg

2.1276840331858002

In [37]:
train2 = train1.fillna(y_avg)

In [38]:
train2.isnull().sum()

Country Name    0
ds              0
y               0
dtype: int64

In [97]:
train1.head()

,Country Name,ds,y
0,Afghanistan,2000,0.055167
1,Afghanistan,2001,0.055293
2,Afghanistan,2002,0.066810
3,Afghanistan,2003,0.073005
4,Afghanistan,2004,0.054867


In [103]:
train1.isnull().sum()

Country Name      0
ds                0
y               432
dtype: int64

In [99]:
train1.shape

(4256, 3)

In [104]:
len(countries)

266

In [40]:
mx = 0
for country in countries:
    tmp = train2[train2['Country Name'] == country]
    no_of_missing = tmp.isnull().sum().sum()
    if no_of_missing > 0:
        print(country, no_of_missing)
    mx = max(mx, no_of_missing)

print('Max missing value', mx)

Max missing value 0


In [14]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3824 entries, 0 to 4255
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  3824 non-null   object 
 1   ds            3824 non-null   int64  
 2   y             3824 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 119.5+ KB


In [15]:
#Final traiend data
train = train2

In [211]:
#train

****

In [82]:
#afg = train[train['Country Name'] == 'Afghanistan'].iloc[:, 1: ]

In [212]:
#afg

In [84]:
#afg.rename(columns = {'year': 'ds', 'CO2 emissions (metric tons per capita)': 'y'}, inplace = True)

In [221]:
#afg

In [86]:
#afg.plot()

In [116]:
#afg['ds'] = pd.to_datetime(afg['ds'], format='%Y')

In [220]:
#afg

In [114]:
#!pip install prophet


In [124]:
#from prophet import Prophet
#model = Prophet()

In [215]:

#model.fit(afg)

In [216]:
#future = model.make_future_dataframe(periods=16, freq='Y') # forecasting 3 years into the future


In [142]:
#forecast = model.predict(future)

In [182]:
#forecast

In [144]:
#forecast['Country Name'] = 'Afghanistan'

In [145]:
#forecast['ds'] = forecast['ds'].dt.year

In [151]:
#new_df = forecast[['Country Name', 'ds', 'yhat']]

In [155]:
#years_of_interest = [2016, 2017, 2018, 2019, 2020, 2030]
#new_df = new_df[new_df['ds'].isin(years_of_interest)]

In [223]:
#new_df

In [173]:
#new_df2 = new_df.pivot(index = 'Country Name', columns = 'ds', values = 'yhat')

In [218]:
#new_df2

In [250]:
#new_df3 = new_df2.reset_index()

In [259]:
#new_df3

In [252]:
#new_df3 = new_df3.drop('Country Name', axis = 1)

In [260]:
#new_df3

In [261]:
#type(new_df3.columns[1])

In [255]:
#new_df3.columns = [f'{col} [YR{col}]' for col in new_df3.columns]

In [262]:
#new_df3

In [257]:
#new_df3.insert(0, 'Country Name', 'Afghanistan')

In [263]:
#new_df3

In [45]:
def buildModelAndPredictions(train, countries):
    predictions_dfs = []
    for country in countries:
        #new_df named country(e.g, Afghanistan,..)
        df = train[train['Country Name'] == country].iloc[:, 1: ] #keeping just two variable: ds, y
        df['ds'] = pd.to_datetime(df['ds'], format='%Y')
        
        
        #Initialize prophet model
        from prophet import Prophet
        model = Prophet()
        
        model.fit(df)
        future = model.make_future_dataframe(periods=16, freq='YE') # forecasting 16 years into the future
        
        #Forecasting
        forecast = model.predict(future)
        
        #Add new column named 'Country Name' into forecast df
        forecast['Country Name'] = country
        forecast['ds'] = forecast['ds'].dt.year
        
        new_df = forecast[['Country Name', 'ds', 'yhat']]
        
        #return new_df
        
        years_of_interest = [2016, 2017, 2018, 2019, 2020, 2030]
        new_df = new_df[new_df['ds'].isin(years_of_interest)]
        
        #return new_df
        
        new_df2 = new_df.pivot(index = 'Country Name', columns = 'ds', values = 'yhat')
        
        #return new_df2
        new_df3 = new_df2.reset_index()
        new_df3 = new_df3.drop('Country Name', axis = 1)
        new_df3.columns = [f'{col} [YR{col}]' for col in new_df3.columns]
        #return new_df3
        new_df3.insert(0, 'Country Name', country)
        
        predictions_dfs.append(new_df3)
    final_predictions_df = pd.concat(predictions_dfs, axis = 0)
    
    #return predictions_dfs
    return final_predictions_df

In [71]:
#train.head(10)

In [72]:
#train.isnull().sum()

**Function Call**

In [43]:
train = train2

In [46]:
output_df = buildModelAndPredictions(train, countries)

15:31:58 - cmdstanpy - INFO - Chain [1] start processing
15:31:58 - cmdstanpy - INFO - Chain [1] done processing
15:31:59 - cmdstanpy - INFO - Chain [1] start processing
15:31:59 - cmdstanpy - INFO - Chain [1] done processing
15:31:59 - cmdstanpy - INFO - Chain [1] start processing
15:32:00 - cmdstanpy - INFO - Chain [1] done processing
15:32:00 - cmdstanpy - INFO - Chain [1] start processing
15:32:00 - cmdstanpy - INFO - Chain [1] done processing
15:32:01 - cmdstanpy - INFO - Chain [1] start processing
15:32:01 - cmdstanpy - INFO - Chain [1] done processing
15:32:02 - cmdstanpy - INFO - Chain [1] start processing
15:32:02 - cmdstanpy - INFO - Chain [1] done processing
15:32:02 - cmdstanpy - INFO - Chain [1] start processing
15:32:02 - cmdstanpy - INFO - Chain [1] done processing
15:32:03 - cmdstanpy - INFO - Chain [1] start processing
15:32:03 - cmdstanpy - INFO - Chain [1] done processing
15:32:03 - cmdstanpy - INFO - Chain [1] start processing
15:32:04 - cmdstanpy - INFO - Chain [1]

In [47]:
df = output_df

In [48]:
df

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,0.410362,0.437392,0.464565,0.491877,0.502442,0.740806
0,Africa Eastern and Southern,0.999340,1.015009,1.036796,1.064660,0.987381,1.000918
0,Africa Western and Central,0.437035,0.434337,0.432505,0.431536,0.423089,0.390666
0,Albania,1.799837,1.754831,1.678996,1.572541,1.972374,2.196606
0,Algeria,4.345719,4.583482,4.847187,5.136641,4.911379,6.544167
...,...,...,...,...,...,...,...
0,West Bank and Gaza,2.127684,2.127684,2.127684,2.127684,2.127684,2.127684
0,World,4.799982,4.859095,4.926573,5.002339,4.876141,5.155047
0,"Yemen, Rep.",0.799184,0.765253,0.734704,0.707561,0.788049,0.701300
0,Zambia,0.287646,0.297987,0.309224,0.321351,0.318974,0.403208


In [49]:
#df.to_csv('Hacktick_58.csv', index = False)

In [83]:
df.shape

(239, 7)

In [85]:
t = test1['Country Name'].unique().tolist()

In [86]:
len(t)

266

In [ ]:
tm = train1['Country Name'] == 

****==============================================================================================****

**Standardization**

In [56]:
#col = 'CO2 emissions (metric tons per capita)'
#mean, sd = np.mean(train[col]), np.std(train[col])

#train[col] = (train[col] - mean / sd)

In [42]:
#train

**Model**

In [84]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [85]:
Y_train = train['CO2 emissions (metric tons per capita)']
X_train = train.drop(['CO2 emissions (metric tons per capita)'], axis = 1)

In [86]:
model.fit(X_train, Y_train)

LinearRegression()

In [60]:
Y_pred = model.predict(test)

**Making Test Dataframe**

In [87]:
years = [2016, 2017, 2018, 2019, 2020, 2030]
country_year = []
for country in countries:
    for year in years:
        s = country + '_' + str(year)
        country_year.append(s)

In [88]:
test_df = pd.DataFrame({'Country Name': country_year})

In [71]:
#test_df

In [89]:

test_df['year'] = test_df['Country Name'].str[-4: ]
test_df['Country Name'] = test_df['Country Name'].str[: -5]
test_df['year'] = pd.to_numeric(test_df['year'])

In [90]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

test_df['Country Name'] = label_encoder.fit_transform(test_df['Country Name'])

In [91]:
predicted_values = model.predict(test_df)
test_df['CO2 Emissions'] = predicted_values
test_df['CO2 Emissions'] = predicted_values
final_df = test_df.pivot(index = 'Country Name', columns = 'year')
final_df.columns = [ f"{col[1]} [YR{col[1]}]"  for col in final_df.columns ]
final_df = final_df.reset_index()

In [92]:
final_df['Country Name'] = countries

In [93]:
final_df

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,1.310290,1.314826,1.319362,1.323899,1.328435,1.373796
1,Africa Eastern and Southern,1.312023,1.316559,1.321095,1.325631,1.330167,1.375528
2,Africa Western and Central,1.313755,1.318291,1.322827,1.327364,1.331900,1.377261
3,Albania,1.315488,1.320024,1.324560,1.329096,1.333632,1.378994
4,Algeria,1.317220,1.321756,1.326293,1.330829,1.335365,1.380726
...,...,...,...,...,...,...,...
261,West Bank and Gaza,1.762480,1.767016,1.771552,1.776088,1.780624,1.825985
262,World,1.764212,1.768748,1.773284,1.777820,1.782357,1.827718
263,"Yemen, Rep.",1.765945,1.770481,1.775017,1.779553,1.784089,1.829450
264,Zambia,1.767677,1.772213,1.776749,1.781286,1.785822,1.831183


In [94]:
#final_df.to_csv('Hacktick_56.csv', index = False)